In [4]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import min, max, upper, trim, udf, col

spark = SparkSession.Builder().appName('Views').getOrCreate()

In [2]:
data = [
    (1, 'Rohit', 1000, 100),
    (2, 'Ajay', 3000, 200),
]

schema = ['id', 'name', 'salary', 'bonus']

df = spark.createDataFrame(data, schema)
df.show()

+---+-----+------+-----+
| id| name|salary|bonus|
+---+-----+------+-----+
|  1|Rohit|  1000|  100|
|  2| Ajay|  3000|  200|
+---+-----+------+-----+



In [9]:
def totalPay(s,b):
    return s+b

TotalPay = udf(lambda x,y : totalPay(x,y), IntegerType())

df.select("*", TotalPay(col('salary'), col('bonus')).alias('TotalPayment')).show()
df.select("*", TotalPay(df.salary, df.bonus).alias('TotalPayment')).show()

+---+-----+------+-----+------------+
| id| name|salary|bonus|TotalPayment|
+---+-----+------+-----+------------+
|  1|Rohit|  1000|  100|        1100|
|  2| Ajay|  3000|  200|        3200|
+---+-----+------+-----+------------+

+---+-----+------+-----+------------+
| id| name|salary|bonus|TotalPayment|
+---+-----+------+-----+------------+
|  1|Rohit|  1000|  100|        1100|
|  2| Ajay|  3000|  200|        3200|
+---+-----+------+-----+------------+



In [ ]:
@udf(returnType=IntegerType())
def totalPay(s,b):
    return s+b

In [11]:
df.select("*", totalPay(col('salary'), col('bonus')).alias('TotalPayment')).show()
df.select("*", totalPay(df.salary, df.bonus).alias('TotalPayment')).show()
df.withColumn('TotalPay', totalPay(df.salary, df.bonus)).show()

+---+-----+------+-----+------------+
| id| name|salary|bonus|TotalPayment|
+---+-----+------+-----+------------+
|  1|Rohit|  1000|  100|        1100|
|  2| Ajay|  3000|  200|        3200|
+---+-----+------+-----+------------+

+---+-----+------+-----+------------+
| id| name|salary|bonus|TotalPayment|
+---+-----+------+-----+------------+
|  1|Rohit|  1000|  100|        1100|
|  2| Ajay|  3000|  200|        3200|
+---+-----+------+-----+------------+

+---+-----+------+-----+--------+
| id| name|salary|bonus|TotalPay|
+---+-----+------+-----+--------+
|  1|Rohit|  1000|  100|    1100|
|  2| Ajay|  3000|  200|    3200|
+---+-----+------+-----+--------+



In [12]:
## Creating a temp view for the table
df.createOrReplaceTempView('employees')

In [14]:
spark.sql(
    """
    SELECT * FROM employees
    """
).show()

+---+-----+------+-----+
| id| name|salary|bonus|
+---+-----+------+-----+
|  1|Rohit|  1000|  100|
|  2| Ajay|  3000|  200|
+---+-----+------+-----+



#### Registering the UDF to spark

In [20]:
spark.udf.register(name='TotalPay', f = totalPay, returnType = IntegerType())

24/06/17 20:49:38 WARN SimpleFunctionRegistry: The function totalpay replaced a previously registered function.


<function __main__.totalPay(s, b)>

In [21]:
spark.sql(
    """
    SELECT *, TotalPay(salary, bonus) as TotalPayment 
    FROM employees
    """
).show()

+---+-----+------+-----+------------+
| id| name|salary|bonus|TotalPayment|
+---+-----+------+-----+------------+
|  1|Rohit|  1000|  100|        1100|
|  2| Ajay|  3000|  200|        3200|
+---+-----+------+-----+------------+



In [1]:
spark.stop()

NameError: name 'spark' is not defined